In [5]:
# Loading libraries
import random
import matplotlib.pyplot as plt             # Library used for plotting
from matplotlib.patches import Rectangle    # Module to plot a rectangle in the image
import urllib.request                       # Library to download data
import numpy as np                          # Library for array manipulation
import seaborn as sn                        # Library for advanced plotting
import pandas as pd                         # Library to manipulate data frames
import tifffile                             # Library to store numpy arrays as TIFF
import pathlib                              # Library to work with file paths
from pathlib import Path                    # Library to work with file paths
import skimage                              # Library for image manipulation. scikit-image
from skimage.io import imread               # Module from skimage
from matplotlib import animation            # Module to plot animations

from ipywidgets import interactive, HBox, VBox, Layout # Library to create interactive widgets
import ipywidgets as widgets               # Library to create interactive widgets

# importing of necessary libraries including cellpose and skimage
from cellpose import plot, models           # Module from cellpose
from skimage.morphology import remove_small_objects # Module from skimage
from skimage.filters import difference_of_gaussians
from skimage import io, measure

In [6]:
img_0 = []
for i in range(6):
    figName = './NoDrug/rep1_FOV' + str(i+1) + '.tif'
    img = imread(str(figName))
    img_0.append(img)
img = img_0[0]

In [67]:
def get_data(img, t=0):
    img_nuc = img[t,:,:,0]
    model = models.Cellpose(model_type='nuclei')
    masks_nuc  = model.eval(img_nuc, diameter=100, channels=[0,1])[0]
    masks_nuc = remove_small_objects(masks_nuc, min_size=50)

    img_cyto = img[t,:,:,1]
    model = models.Cellpose(model_type='cyto')
    masks_cyto = model.eval(img_cyto, diameter=200, channels=[0,2])[0]

    nucleus_indices = np.zeros(np.max(masks_cyto)+1)
    for i in range(1,np.max(masks_nuc)+1):
        posn_nucl = np.mean(np.where(masks_nuc==i),axis=1).astype(int)
        nucleus_indices[masks_cyto[posn_nucl[0],posn_nucl[1]]] = i
    
    number_detected_cells = np.max(masks_cyto)

    list_cyt_areas = []
    list_cyt_intensities = []
    list_nuc_areas = []
    for i in range (1,number_detected_cells+1):
        selected_cyto_mask = (masks_cyto==i)
        area_cyto = np.sum(selected_cyto_mask)
        list_cyt_areas.append(area_cyto)
        selected_color_image = selected_cyto_mask*img_cyto
        list_cyt_intensities.append(selected_color_image[np.nonzero(selected_color_image)].mean())
        
        selected_nuc_mask = (masks_nuc==nucleus_indices[i])
        area_nuc = np.sum(selected_nuc_mask)
        list_nuc_areas.append(area_nuc)

    print('Nucleus and cytoplast areas and intensities done!')

    ### mRNA pre-processing
    img_spots = img[t,:,:,2]
    img_spots_filtered = difference_of_gaussians(img_spots,low_sigma=1, high_sigma=5)
    threshold = 0.05
    img_spots_binary = img_spots_filtered.copy()
    img_spots_binary[img_spots_binary<threshold] = 0 # Making spots below the threshold equal to 0.
    img_spots_binary[img_spots_binary>=threshold] = 1 # Making spots above the threshold equal to the threshold value.
    spot_contours = measure.find_contours(img_spots_binary, 0.5)

    ### counting mRNA particles
    list_cytosol_particles = np.zeros(number_detected_cells)
    list_nuclear_particles = np.zeros(number_detected_cells)
    # Loop through the cytosol masks
    for i in range(len(spot_contours)):
        # Find the position of the particle
        posn = np.mean(spot_contours[i], axis=0).astype(int)

        # Check which cyto mask is the particle in
        cell_num_nuc = masks_nuc[posn[0], posn[1]]
        cell_num_cyto = masks_cyto[posn[0], posn[1]]
        if cell_num_nuc>0:
             list_nuclear_particles[cell_num_nuc-1] += 1
        elif cell_num_cyto>0:
            list_cytosol_particles[cell_num_cyto-1] += 1

    print('Number of mRNA done!')

    ### transcription sites pre-processing
    threshold = 50
    img_site = img[t,:,:,2]
    img_sites_binary = img_site.copy()
    img_sites_binary[img_sites_binary<threshold] = 0
    img_sites_binary[img_sites_binary>=threshold] = 1 # Making spots above the threshold equal to the threshold value.
    site_contours = measure.find_contours(img_sites_binary, 0.5)

    list_site_particles = np.zeros(number_detected_cells)
    for i in range(len(site_contours)):
        # Find the position of the particle
        posn = np.mean(site_contours[i], axis=0).astype(int)

        # Check which cyto mask is the particle in
        cell_num = masks_nuc[posn[0], posn[1]]
        if cell_num>0:
            list_site_particles[cell_num-1] += 1
    
    print('Transcription site number done!')
    
    list_site_intensities = np.zeros(number_detected_cells)
    selected_mask = (img_sites_binary > 0)
    for i in range (1, number_detected_cells+1):
        selected_nuc_mask = (masks_nuc == i)
        selected_color_image = selected_mask*selected_nuc_mask*img_spots
        selected_nonzero = selected_color_image[np.nonzero(selected_color_image)]
        if selected_nonzero.size > 0:
            list_site_intensities[i-1] = selected_nonzero.mean()
    print('Transcription site intensities done!')

    df = pd.DataFrame(list(range(1,number_detected_cells+1)), columns=['cell index'])
    df['time'] = np.array([t for i in range(number_detected_cells)])
    df['nucleus area'] = np.array(list_nuc_areas).astype(int)
    df['cytosol area'] = np.array(list_cyt_areas).astype(int)
    df['protein intensity'] = list_cyt_intensities
    df['mRNA in cytosol'] = list_cytosol_particles.astype(int)
    df['mRNA in nucleus'] = list_nuclear_particles.astype(int)
    df['site in nucleus'] = list_site_particles.astype(int)
    df['site intensity'] = list_site_intensities

    print('Summary of dataframe done!')

    return df

In [68]:
df = get_data(img,0)

Nucleus and cytoplast areas and intensities done!
Number of mRNA done!
Transcription site number done!
Transcription site intensities done!
Summary of dataframe done!


In [69]:
df

,cell index,time,nucleus area,cytosol area,protein intensity,mRNA in cytosol,mRNA in nucleus,site in nucleus,site intensity
0,1,0,2748,8158,81.343834,6,3,3,53.369565
1,2,0,2769,9572,63.891245,0,0,0,0.000000
2,3,0,2830,9944,121.761062,15,6,8,92.603093
3,4,0,2971,9393,88.646652,8,3,3,52.676471
4,5,0,2655,8561,109.587548,9,3,4,85.256039
5,6,0,2959,9947,117.721524,13,2,2,52.750000
6,7,0,2636,8824,66.981981,0,1,1,80.970238
7,8,0,2568,8986,81.629312,1,0,0,0.000000
8,9,0,2941,8879,77.287532,2,0,0,0.000000
9,10,0,2756,11206,124.436820,15,6,7,88.882562
